### В рамках данной работы проверены следующие подходы и получиены следующие результаты:
- Байзлайн: епсилон-жадный. Регрет = 2792
- Полная случайность. Регрет = 17052
- Бернулли Томсон-семплинг. Регрет = 1471 (ниже бейзлайна)
- Upper confidence bound. Регрет = 4339

In [2]:
!pip install tqdm

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 76 kB 638 kB/s eta 0:00:01
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import pandas as pd
import time

from functools import partial
from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None


### Байзлайн: епсилон-жадный. Регрет = 2792

In [3]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.08)

In [4]:
# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


336.886842250824

In [5]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(2792.237649427154, 0.01396118824713577, 174)

In [6]:
output['history']

,impressions,clicks,lifetime,p
132,19843.0,1332.0,10870.812904,0.064972
162,154.0,2.0,18594.827945,0.017514
163,86.0,0.0,5153.010937,0.000849
164,68.0,2.0,5092.571727,0.041281
166,48.0,0.0,5340.552070,0.007253
167,33.0,0.0,1474.181162,0.033849
168,23.0,0.0,4900.260295,0.027273
169,14.0,0.0,5007.022458,0.030857
170,18.0,0.0,8920.324215,0.034653
171,14.0,0.0,1080.025985,0.012549


### Полная случайность. Регрет = 17052

In [2]:
from random import randrange

def random(history: pd.DataFrame):
    n = history.shape[0]
    return history.index[randrange(n)]

policy = partial(random)

In [3]:
# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

  0%|                                     | 136/200000 [00:00<04:52, 682.63it/s]

1 impressions have been simulated


 50%|█████████████████                 | 100162/200000 [02:09<02:04, 801.43it/s]

100001 impressions have been simulated


100%|██████████████████████████████████| 200000/200000 [04:19<00:00, 770.90it/s]


259.4541161060333

In [4]:
# random regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(17052.32107502962, 0.0852616053751481, 185)

In [6]:
output['history']

,impressions,clicks,lifetime,p
169,2329.0,38.0,12570.660923,0.014629
173,1356.0,107.0,2590.948318,0.077850
174,1234.0,83.0,1630.594595,0.059940
177,868.0,86.0,1959.841208,0.109108
178,775.0,31.0,18537.925424,0.055495
181,425.0,2.0,660.380946,0.005876
182,407.0,9.0,3253.772281,0.032968
184,72.0,0.0,3025.997858,0.008220


### Бернулли Томсон-семплинг. Регрет = 1471

In [10]:
def ts(history: pd.DataFrame):
    betas = np.random.beta(history['clicks'] + 1, history['impressions'] - history['clicks'] + 1)
    return history.index[np.argmax(betas)]

policy = ts

In [11]:
# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

  0%|                                     | 119/200000 [00:00<05:34, 597.89it/s]

1 impressions have been simulated


 50%|█████████████████                 | 100081/200000 [02:42<02:45, 603.86it/s]

100001 impressions have been simulated


100%|██████████████████████████████████| 200000/200000 [05:24<00:00, 615.52it/s]


324.93260192871094

In [12]:
# ts regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1471.230717361168, 0.00735615358680584, 179)

### Upper confidence bound. Регрет = 4339

In [34]:
def ucb(history: pd.DataFrame):
    ctr = history['clicks']  / (history['impressions'] + 10)
    bound = ctr + (np.sqrt(2 * (np.log(len(history) + 1)) / (history['impressions'] + 1)))
    return history.index[np.argmax(bound)]

policy = partial(ucb)

In [35]:
# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

  0%|                                     | 101/200000 [00:00<06:39, 500.96it/s]

1 impressions have been simulated


 50%|█████████████████                 | 100095/200000 [03:10<03:14, 514.59it/s]

100001 impressions have been simulated


100%|██████████████████████████████████| 200000/200000 [06:18<00:00, 528.13it/s]


378.7000000476837

In [36]:
# ucb regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(4339.679392214294, 0.02169839696107147, 185)